In [1]:
seed <- 42

n.genes <- 2000
n.pcs <- 20

use_agf <- TRUE

set.seed(seed)

In [2]:
suppressPackageStartupMessages({
    library(SpatialExperiment)
})

In [3]:
sample.dir <- "data/LIBD_DLPFC/Br8100_151673"

out.dir <- "results/Banksy/Br8100_151673"

dir.create(out.dir, showWarnings = FALSE, recursive = TRUE)

In [4]:
get_SpatialExperiment <- function(
    feature_file,
    observation_file,
    coord_file,
    matrix_file,
    assay_name = "counts") {
    rowData <- read.delim(feature_file, stringsAsFactors = FALSE, row.names = 1, numerals = "no.loss")
    colData <- read.delim(observation_file, stringsAsFactors = FALSE, row.names = 1, numerals = "no.loss")

    coordinates <- read.delim(coord_file, sep = "\t", row.names = 1, numerals = "no.loss")
    coordinates <- as.matrix(coordinates[rownames(colData), ])
    mode(coordinates) <- "numeric"

    spe <- SpatialExperiment::SpatialExperiment(
        rowData = rowData, colData = colData, spatialCoords = coordinates
    )

    assay(spe, assay_name, withDimnames = FALSE) <- as(Matrix::t(Matrix::readMM(matrix_file)), "CsparseMatrix")

    return(spe)
}

In [5]:
spe <- get_SpatialExperiment(
    feature_file = file.path(sample.dir, "features.tsv"),
    observation_file = file.path(sample.dir, "observations.tsv"),
    coord_file = file.path(sample.dir, "coordinates.tsv"),
    matrix_file = file.path(sample.dir, "counts.mtx")
)

In [7]:
counts <- assay(spe, "counts")

seu <- Seurat::CreateSeuratObject(counts = counts)
seu <- Seurat::NormalizeData(seu, normalization.method = "RC", scale.factor = median(colSums(counts)), verbose = FALSE)
seu <- Seurat::FindVariableFeatures(seu, nfeatures = n.genes, verbose = FALSE)

spe <- spe[Seurat::VariableFeatures(seu), ]

In [8]:
spe <- scuttle::computeLibraryFactors(spe)
assay(spe, "normcounts") <- scuttle::normalizeCounts(spe, log = FALSE)

In [9]:
spe <- Banksy::computeBanksy(
    spe,
    assay_name = "normcounts",
    k_geom = 18,
    compute_agf = use_agf,
    coord_names = c("row", "col"),
    verbose = FALSE,
    seed = seed,
)

In [10]:
lambdas <- seq(0, 0.3, 0.1)

for (i in seq_along(lambdas)) {
    lambda <- lambdas[i]

    spe <- Banksy::runBanksyPCA(spe, lambda = lambda, npcs = n.pcs, use_agf = use_agf, seed = seed)

    file <- file.path(out.dir, paste0("banksy_l", as.character(lambda), ".tsv"))
    write.table(reducedDims(spe)[[i]], file = file, sep = "\t", col.names = NA, quote = FALSE)
}

Using seed=42

Using seed=42

Using seed=42

Using seed=42

